# Convert  ".root" data into ".pkl"

In [2]:
%pylab inline

import uproot
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


ImportError: No module named uproot

## Run the filtering macros

In [5]:
import ROOT
from ROOT import gROOT

Welcome to JupyROOT 6.10/08


In [ ]:
gROOT.ProcessLine(".x ../TreeGeneration/HyperTreeFatherData.cc")

In [6]:
gROOT.ProcessLine(".x ../TreeGeneration/HyperTreeFatherMC.cc")

0L

 *** TDatabasePDG::AddParticle: particle with PDGcode=3124 already defined
 *** TDatabasePDG::AddAntiParticle: can't redefine parameters


Error in <GSLError>: Error 18 in qags.c at 548 : cannot reach tolerance because of roundoff error
Warning in <TF1::IntegralOneDim>: Error found in integrating function BlastWave in [0.000000,10.000000] using AdaptiveSingular. Result = 0.000237 +/- 0.000000  - status = 18
Info in <TF1::IntegralOneDim>: 		Function Parameters = { mass =  2.809230 , beta_max =  0.872810 , T =  0.090000 , n =  0.725529 , norm =  71366555458.970901 }
Error in <GSLError>: Error 18 in qags.c at 548 : cannot reach tolerance because of roundoff error
Warning in <TF1::IntegralOneDim>: Error found in integrating function BlastWave in [0.000000,10.000000] using AdaptiveSingular. Result = 0.000155 +/- 0.000000  - status = 18
Info in <TF1::IntegralOneDim>: 		Function Parameters = { mass =  2.809230 , beta_max =  0.843838 , T =  0.116299 , n =  1.043131 , norm =  29265618.006672 }
Warning in <TClass::Init>: no dictionary for class ROOT::TIOFeatures is available


## Loading data and MC samples

In [ ]:
df_event=uproot.open('../../../HypertritonAnalysis/Trees/HyperTritonTree_18r.root')['_custom']['fTreeV0'].pandas.df('fCent')

In [ ]:
Nev_1040=len(df_event.query('10<fCent<=40'))

In [ ]:
df_MC = uproot.open('../../../HypertritonAnalysis//Derived_Trees/HyperTree_MC_Signal.root')['HyperTree_MC_Signal'].pandas.df()

In [ ]:
df_data = uproot.open('../../../HypertritonAnalysis/Derived_Trees/HyperTree_Data.root')['HyperTree_Data'].pandas.df()

## Calculate pre selection efficiency

In [ ]:
cuts_presel='ProngsDCA<1.6 and He3ProngPvDCA>0.01 and He3ProngPvDCA>0.01 and V0CosPA>0.99 and Centrality<10'

In [ ]:
eff_v0=[0.23,0.32,0.42,0.51]

In [ ]:
eff_23=eff_v0[0]*len(df_MC.query(cuts_presel+' and 2<V0pt<3'))/len(df_MC.query('2<V0pt<3 and Centrality<10'))

In [ ]:
eff_34=eff_v0[1]*len(df_MC.query(cuts_presel+' and 3<V0pt<4'))/len(df_MC.query('3<V0pt<4 and Centrality<10'))

In [ ]:
eff_45=eff_v0[2]*len(df_MC.query(cuts_presel+' and 4<V0pt<5'))/len(df_MC.query('4<V0pt<5 and Centrality<10'))

In [ ]:
eff_59=eff_v0[3]*len(df_MC.query(cuts_presel+' and 5<V0pt<9'))/len(df_MC.query('5<V0pt<9 and Centrality<10'))

In [ ]:
eff_presel=[eff_23,eff_34,eff_45,eff_59]

In [ ]:
np.save("../PreSelEfficiency/eff_presel_0_10",eff_presel)

## ML dataframe 10-40

In [ ]:
df_MC=df_MC.query(cuts_presel)

In [ ]:
df_Bkg = df_data.query('(InvMass<2.98 or InvMass>3.005) and V0pt<=10')
df_Sig_data=df_data.query('2.960<InvMass<3.050 and V0pt<=10')

In [ ]:
df_Bkg['y']=0
df_MC['y']=1
df_ML= pd.concat([df_MC,df_Bkg.query('Centrality<10')])
df_ML.to_pickle('../../../HypertritonAnalysis/Dataframes/df_ML.pkl')

In [ ]:
df_Sig_data.query('Centrality<10').to_pickle('../../../HypertritonAnalysis/Dataframes/df_Sig_data.pkl')